This notebook is written by the DAWS2 UQ team based in Univ. of Liverpool for the workshop of UQ & M of the DAWS2 project.

Uncertainty Quantification & Management
--
*placeholder texts*

{

Personal website: https://yuchenakaleslie.github.io/<br>
ResearchGate: https://www.researchgate.net/profile/Yu-Chen-402<br>
Contact: yu.chen2@liverpool.ac.uk

}

DAWS2 UQ team: {Scott Ferson, Ioanna,  [(Leslie) Yu Chen](https://yuchenakaleslie.github.io/)}

All rights reserved.

===================

## Table of Contents
#### [1. Setup](#Setup)
#### [2. Problem statement](#problem_statement)
#### [3. Create propagating function](#create_propagating_function)
#### [4. Epistemic uncertainty](#propagate_epistemic_uncertainty)
#### <small>[4.1. Uncertainty Characterisation](#4.1-uncertainty-characterisation)</small>
#### <small>[4.2. Uncertainty propagation methods](#4.2-uncertainty-propatation-methods)</small>
##### <small>[4.2.1. Endpoints propagation](#4.2.1-Endpoints-propatation)</small>
##### <small>[4.2.2. Subinterval reconstitution propagation](#4.2.2.-Subinterval-reconstitution-propagation)</small>
##### <small>[4.2.3. Sampling propagation](#4.2.3.-Sampling-propagation)</small>
##### <small>[4.2.4. Optimisation methods](#4.2.3.-Optimisation-methods)</small>
#### [5. Aleatory uncertainty](#aleatory_uncertainty_propagation)
#### [5. Mixed types of uncertainty](#mixed_uncertainty_propagation)

***
<a id="setup"></a>
# 1. Setup

Import the libraries

In [1]:
import numpy as np
from PyUncertainNumber.UP.uncertaintyPropagation import epistemic_propagation
from PyUncertainNumber import UncertainNumber as UN
import matplotlib.pyplot as plt

In [2]:
%load_ext autoreload
%autoreload 2
%load_ext rich

In [3]:
# import user-defined function

USER_dir = 'C:\\Users\\Ioanna\\Documents\\GitHub\\PyUncertainNumber\\cantilever_beam'
import sys
sys.path.append(USER_dir)

***
<a id="problem_statement"></a>
# 2. Problem statement

The numerical example models a simple cantilever beam with length, $L$, distance to the neutral axis $y$, Young’s modulus, $E$, second moment of inertia, $I$, and external load, $F$. We will compute the bending stress $σ$, and the deflection $d$, assuming the above input parameters are **uncertain numbers**.

We will use this example throughout the document to illustrate the differences among various uncertainty quantification approaches. A complete description of the following methods can be found in the [DAWS1 report](https://sites.google.com/view/dawsreports/up).

![alt text](../assets/cantilever.png)

**Fig.1** Cantilever beam with input parameters.  

***
<a id="create_propagating_function"></a>
# 3. Create the propagating function


A propagating function is created which calculates the deflection, $d$, as a function of $L$, $E$, $I$, and $F$.
- The function's input is only the uncertain numbers.
- As sampling techniques are used it is likely that for a certain input combination the airfoil will fail to reach a solution. we use the try function to accomodate for this.


Alternatively, such propating function can be imported as a Python function/method object from a certain local directory.

<a id="3.1.-create-function"></a>

## 3.1. Create propagating function

In [3]:
def cantilever_beam_deflection(x):
    """Calculates deflection and stress for a cantilever beam.

    Args:
        x (np.array): Array of input parameters:
            x[0]: Length of the beam (m)
            x[1]: Second moment of area (mm^4)
            x[2]: Applied force (N)
            x[3]: Young's modulus (MPa)

    Returns:
        float: deflection (m)
               Returns np.nan if calculation error occurs.
    """

    beam_length = x[0]
    I = x[1]
    F = x[2]
    E = x[3]
    try:  # try is used to account for cases where the input combinations leads to error in fun due to bugs
        deflection = F * beam_length**3 / (3 * E * 10**6 * I)  # deflection in m
        
    except:
        deflection = np.nan

    return deflection

In [ ]:
# know the function with default Python help mechanism
cantilever_beam_deflection?

In [ ]:
import numpy as np 
x = np.array([1.00500000e+01 , 3.86159100e-04 ,-8.24387293e+00 , 2.12872340e+02])
y = cantilever_beam_deflection(x)
print(y)

<a id="3.2.-Upload-function"></a>

## 3.2. Upload propagating function

In [7]:
# Alternatively, such propating function can be imported as a Python function/method object from a certain local directory.
from PyUncertainNumber.UP.performance_func import cb_deflection, cb_stress

<a id="3.3.-verify-function"></a>

## 3.3. Verify the function

To ensure that the function yields meaningful results. We consider that input has the nominal values seen below.

The function should yield deflection equal to 0.162m.
 

In [ ]:
# test the function

y  = 0.155 # m
L  = 10.05 # m
I = 0.000386 # m**4
F = 37 # kN
E = 200 # GPa

x = np.array([L, I, F, E])
deflection = cantilever_beam_deflection(x)

print(deflection) # 0.162m

***
<a id="propagate_epistemic_uncertainty"></a>
# 4. Epistemic uncertainty

<a id="4.1.-uncertainty-characterisation"></a>

## 4.1. Uncertainty Characterisation

Construct `UncertainNumbers` objects for the inputs assuming they are intervals with the lower and upper values are seen below

- $y = [0.145, 0.155] \ m$

- $L = [9.95, 10.05] \ m$

- $I = [0.0003861591, 0.0005213425] \ m^{4}$

- $F = [11, 37] \ kN$

- $E = [200, 220] \ GPa$
 

In [2]:
#y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) # Required only for stress estimation
L = UN(name='beam length', symbol='L', units='m', essence='interval', bounds= [9.95, 10.05])
I = UN(name='moment of inertia', symbol='I', units='m', essence='interval', bounds= [0.0003861591, 0.0005213425])
F = UN(name='vertical force', symbol='F', units='kN', essence='interval', bounds= [11, 37])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='interval', bounds=[200, 220])

In [ ]:

L.display()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

def plot_interval_box(lower_bound, upper_bound):
    """Plots an interval as a box on the y-axis at y=1 with no x-axis numbers 
       and includes vertical lines from y=0 to y=1 at the bounds.
    """

    plt.figure(figsize=(3, 3))  # Adjust figure size as needed

    # Create the box
    
    plt.hlines(y=1, xmin=lower_bound, xmax=upper_bound, linewidth=2, color='blue')

    # Add vertical lines
    plt.vlines(x=lower_bound, ymin=0, ymax=1, linewidth=1, color='blue')
    plt.vlines(x=upper_bound, ymin=0, ymax=1, linewidth=1, color='blue')

    #plt.yticks([0])  # Show only the y-axis tick at 0 
    plt.xticks([])  # Remove x-axis ticks and numbers

    plt.xlabel("V")
    plt.ylabel("CDF")

    plt.show()


# Example usage:
plot_interval_box(2, 5)

<a id="4.2-uncertainty-propatation-methods"></a>

## 4.2 Ucertainty propagation methods 

Choose from a suite of black box propagating techniques to propagate the intervals through the model. 

<a id="4.2.1.-Endpoints-propatation"></a>

### 4.2.1. Endpoint propagation

The endpoint propagation method (Dong and Shah, 1987) is a straightforward way to project intervals through the code, by projecting all input combinations produced by the Cartesian product of the interval bounds. This results in a total of $n = 2^{d}$. 

For the working example, there are d = 5 intervals which results in $n = 2^{5} = 32$ input combinations.

#### Assumptions




- [x] Ioanna's original top-level UP function with  the 'endspoints' method

- [x] Leslie's standalone endspoints implementation

In [ ]:
''' works perfect '''

# deflection_bounds_e = UN.endpointsMethod(vars=['L', 'I', 'F', 'E'], 
#                                          func = cb_deflection, 
#                                          name='deflection', 
#                                          symbol='D')
# deflection_bounds_e

# stress_bounds_e = UN.endpointsMethod(vars=['y', 'L', 'I', 'F'], 
#                                      func= cb_stress, 
#                                      name='stress', 
#                                      symbol='S')
# stress_bounds_e

- [x] Leslie's integrated top-lelvel endspoints implementation

In [ ]:
METHOD = "endpoint"

a = epistemic_propagation(vars=['L', 'I', 'F', 'E'], 
          fun=cantilever_beam_deflection, 
          n = None, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = USER_dir
         )

In [ ]:
# returned a dict that contains the UN object and some other information
a.keys()

In [ ]:
a["un"]

<a id="4.2.2.-Subinterval-reconstitution-propagation"></a>

### 4.2.2. Subinterval reconstitution propagation

The input intervals are partitioned into smaller intervals, which are then propagated through the model using endpoint propagation and the output interval can be reassembled (Ferson and Hajagos, 2004).

In [ ]:
METHOD = "subinterval"
base_path = ""

# TODO return either as namedTuple or dict to be explicit
a = epistemic_propagation(vars=['L', 'I', 'F', 'E'], 
          fun = cantilever_beam_deflection, 
          n = 10, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = base_path)

In [ ]:
a.keys()

In [ ]:
    
A = UN(
    essence="pbox",
    distribution_parameters=["gaussian", [[1, 2], [1,2]]]
)

B = UN(
    essence="distribution",
    distribution_parameters=["gaussian", [1, 1]]
)

C = UN(
    essence="interval",
    bounds= [3, 4] 
)

V = UN(name='testing', 
        symbol='V', 
        essence='distribution', 
        distribution_parameters=["triang", [0.5, 10,20]])

C.display
plt.show
# print(A)
# print(B)
# print(C)

# # example
# from PyUncertainNumber import UncertainNumber as UN

def cantilever_beam_deflection(x):
    """Calculates deflection and stress for a cantilever beam.

    Args:

        x (np.array): Array of input parameters:
            x[0]: Length of the beam (m)
            x[1]: Second moment of area (mm^4)
            x[2]: Applied force (N)
            x[3]: Young's modulus (MPa)

    Returns:
        float: deflection (m)
               Returns np.nan if calculation error occurs.
    """

    beam_length = x[0]
    I = x[1]
    F = x[2]
    E = x[3]
    try:  # try is used to account for cases where the input combinations leads to error in fun due to bugs
        deflection = F * beam_length**3 / (3 * E * 10**6 * I)  # deflection in m
        
    except:
        deflection = np.nan

    return np.array([deflection]) 

#L = UN(name='beam length', symbol='L', units='m', essence='distribution', distribution_parameters=["gaussian", [10.05, 0.033]])
#I = UN(name='moment of inertia', symbol='I', units='m', essence='distribution', distribution_parameters=["gaussian", [0.000454, 4.5061e-5]])
F = UN(name='vertical force', symbol='F', units='kN', essence='distribution', distribution_parameters=["gaussian", [24, 8.67]])
#I = UN(name='moment of inertia', symbol='I', units='m', essence='interval', bounds=["gaussian", [0.000454, 4.5061e-5]])

E = UN(name='elastic modulus', symbol='E', units='GPa', essence='distribution', distribution_parameters=["gaussian", [210, 6.67]])
L = UN(name='beam length', symbol='L', units='m', essence='interval', bounds= [9.95, 10.05])
I = UN(name='moment of inertia', symbol='I', units='m', essence='interval', bounds= [0.0003861591, 0.0005213425])
#F = UN(name='vertical force', symbol='F', units='kN', essence='interval', bounds= [11, 37])
#E = UN(name='elastic modulus', symbol='E', units='GPa', essence='interval', bounds=[200, 220])
#E.display()
#plt.show()
#  print(V)

# print(L.essence)
# METHOD = "latin_hypercube"
# base_path = ""

a = first_order_propagation_method(x=[L, I, F, E], #['L', 'I', 'F', 'E'], 
          f = cantilever_beam_deflection, 
          n_disc= 3
         )

print(a['bounds'])
# # Create a p-box UncertainNumber with a Gaussian distribution
R = UN(name='deflection', symbol='d', units='m', essence='pbox', distribution_parameters=['empirical', a['bounds']])

F = UN(name='vertical force', symbol='F', units='kN', essence='pbox', distribution_parameters=["gaussian", [[22.5, 26.5], 8.67]])

#F.display()
I = UN(name='moment of inertia', symbol='I', units='m', essence='pbox', distribution_parameters=["gaussian", [[0.0035,0.00067], 4.5061e-5]])

#I.display()

# Calculate the positions for the horizontal lines (3 equally spaced points)
y_positions = np.linspace(0, 1, 5)  # 4 points create 3 spaces
print(y_positions)
# Plot the horizontal lines
for y in y_positions[1:]:  # Skip the first point (ymin)
    plt.axhline(y=y, color='gray', linestyle='--')

#plt.show()
# res = endpoints_monotonic_method(ranges.T, f)

    # num_outputs = res['sign_x'].shape[0]
    # inpsList = np.zeros((0, d))
    # evalsList = np.zeros((0, num_outputs))  # for any number fo outputs
    # X_input = []

    # X_signs = np.empty((num_outputs, d, n_slices.sum(), 2, d))

    # for out in range(num_outputs):
    #     for input in range(d):
    #         X = [ranges[:,k].tolist() for k in range(d)]
    #         temp_X = X.copy()
    #         temp_X[input] = []
    #         Xsings = np.empty((n_slices[input], 2, d))
            
    #         current_index = 0
    #         for slice in range(n_slices[input]):
    #             temp_X[input] = []

    #             temp_X[input].extend(np.array([xl[input][slice], xr[input][slice]]).tolist())
    #             rang = np.array([temp_X[i] for i in range(d)], dtype=object)
    #             Xsings[slice, :, :] = extreme_pointX(rang, res['sign_x'][out])
    #             X_signs[out, input, current_index, :, :] = Xsings[slice, :, :]  # Store in X_signs
    #             current_index += 1
                
    #         print(out, input, Xsings)  # Print for debugging
    #             #print('temp_X', out, input, slice, temp_X)  # Print for debugging



    #     # Initialize all_output as a list to store outputs initially
    #             all_output_list = []
    #             evalsList = []
    #             numRun = 0
    #             inpsList = np.empty((0, x_combinations.shape[1]))  # Initialize inpsList with the correct number of columns

    #             for case in x_combinations:  # Iterate directly over the rows of x_combinations
    #                 im = np.where((inpsList == case).all(axis=1))[0]
    #                 if not im.size:
    #                     output = f(case)
    #                     all_output_list.append(output)
    #                     inpsList = np.vstack([inpsList, case])
    #                     evalsList.append(output)
    #                     numRun += 1
    #                 else:
    #                     all_output_list.append(evalsList[im[0]])



    # for input in range(d):     
        
    #     X = [ranges[:,i].tolist() for i in range(d)]
    #     temp_X = X.copy()  # Create a temporary copy of X
    #     temp_X[input] = []  # Clear the list for the current input
    #     Xsings = np.empty((num_outputs, n_slices[input], 2, d))

    #     current_index = 0
    #     for slice in range(n_slices[input]):
    #         temp_X[input].extend(np.array([xl[input][slice], xr[input][slice]]).tolist())  # Extend the list for the current input
        
    #     rang = np.array([temp_X[i] for i in range(d)], dtype=object)
         
    #     for out in range(num_outputs):
    #         Xsings[out, current_index, :, :] = extreme_pointX(rang, res['sign_x'][out])
    #         current_index += 1  
    #     print(input, Xsings)
    #     print('temp_X', input, temp_X)

    #    # X_input.append(temp_X)

        

    

        # Generate focal_elemefocal_elements_comb = cartesian(*[np.arange(slice) for i in range(d)])nts_comb with correct cartesian input
        

    #print('focal elements', focal_elements_comb)

    # # Create a list of lists containing both lower and upper bounds for each number
    # # combined_bounds = [
    # #         np.concatenate((np.atleast_1d(lower), np.atleast_1d(upper)))
    # #         for lower, upper in zip(xl, xr)
    # #             ]
    # # # Generate the Cartesian product of all combinations
    # # all_combinations = cartesian(*combined_bounds)
    # # #print(all_combinations)

    # all_output = None
    # if f is not None:
    #     # Efficiency upgrade: store repeated evaluations
    #     inpsList = np.zeros((0, d))
    #     evalsList = []  
    #     numRun = 0

    #     all_output = []  
    #     for c in tqdm.tqdm(all_combinations, desc="Evaluating samples"):
    #         im = np.where((inpsList == c).all(axis=1))[0]
    #         if not im.size:
    #             output = f(c) 
    #             all_output.append(output)
    #             inpsList = np.vstack([inpsList, c])
    #             evalsList.append(output)
    #             numRun += 1
    #         else:
    #             all_output.append(evalsList[im[0]]) 
        
    #     # Determine the number of outputs from the first evaluation
    #     try:
    #         num_outputs = len(all_output[0])
    #     except TypeError:
    #         num_outputs = 1  # If f returns a single value

    #     # Convert all_output to a NumPy array with the correct shape
    #     all_output = np.array(all_output).reshape(-1, num_outputs)  

    # return all_combinations, all_output #Include all_combinations in the return values
    
   

    # xl = np.zeros((nd , d))
    # xr = np.zeros((nd, d))
    # ranges = np.zeros((2, d))
    
    # for i, un in enumerate(x):
    #     match un.essence:
            
    #         case "distribution":
    #             # xl[:, i] = un.ppf(u[:-1])  # Calculate ppf for all quantiles at once
    #             # print('xl', xl[:,i])
    #             # xr[:, i] = un.ppf(u[1:])   # Calculate ppf for all quantiles at once
    #             # ranges[:, i] = [xl[0, i], xr[-1, i]]
    #             pass
    #         case 'interval':
    #             # Assign the lower bound to the first row of xl for this UN
    #             xl[0, i] = un.bounds[0]  
    #             # Assign the upper bound to the last row of xr for this UN
    #             xr[-1, i] = un.bounds[1]  
    #             ranges[:, i] = un.bounds
    #         case "pbox":
    #             print("pbox_ppf", un.ppf(u))
    #             xl[:, i] = un.ppf(u)[0]  # Calculate ppf for all quantiles at once
    #             xr[:, i] = un.ppf(u)[1]   # Calculate ppf for all quantiles at once
    #             ranges[:, i] = [xl[0, i], xr[-1, i]]
    #         case _:
    #             raise ValueError(f"Unsupported uncertainty type: {un.essence}")

    # return xl, xr, ranges
    #xl, xr, range = mixed_uncertainty(x =[A,B,C], f= None,   n_disc=3)


#print(xr)
#print(range)

    # # Efficiency upgrade: store repeated evaluations
    # inpsList = np.zeros((0, d))
    # evalsList = np.zeros((0, 2))
    # numRun = 0

    # # Generate Cartesian product and propagate
    # clD = np.full((nd - 1, 2, d), np.nan)
    # cmD = np.full((nd - 1, 2, d), np.nan)
    # for i in range(d):  # First-order propagation
    #     X = ranges.copy()
    #     for j in range(nd - 1):
    #         X[:, i] = [xl[j, i], xr[j, i]]

    #         # Extreme point method
    #         XsignCl = extreme_pointX(X, signs[:, 0])
    #         XsignCm = extreme_pointX(X, signs[:, 1])
    #         C = np.vstack([XsignCl, XsignCm])

    #         cl = np.full(C.shape[0], np.nan)
    #         cm = np.full(C.shape[0], np.nan)
    #         for k in range(C.shape[0]):
    #             c = C[k]
    #             print(f'Input: {i + 1}, FE: {j + 1}, run: {k + 1} -->  ', end='')

    #             # Check for existing evaluations
    #             im = np.where((inpsList == c).all(axis=1))[0]
    #             if not im.size:
    #                 # Replace the following line with your actual xfoilrun function call
    #                 cl[k], cm[k] = simulate_xfoilrun(c)  
    #                 inpsList = np.vstack([inpsList, c])
    #                 evalsList = np.vstack([evalsList, [cl[k], cm[k]]])
    #                 numRun += 1
    #                 print(f'number of evaluations: {numRun}')
    #             else:
    #                 cl[k] = evalsList[im[0], 0]
    #                 cm[k] = evalsList[im[0], 1]
    #                 print('evaluation exists - skipping...')

    #         clD[j, :, i] = [np.min(cl), np.max(cl)]
    #         cmD[j, :, i] = [np.min(cm), np.max(cm)]
    #         print()

    #     clD[:, :, i] = np.array(sorted(clD[:, :, i], key=lambda x: x[0]))  # Sort by first column
    #     cmD[:, :, i] = np.array(sorted(cmD[:, :, i], key=lambda x: x[0]))
    #     print('\n\n')

    # clI = np.zeros((nd - 1, 2))
    # cmI = np.zeros((nd - 1, 2))
    # for i in range(nd - 1):
    #     clI[i] = imp(np.transpose(clD[i, :, :], (1, 0)))
    #     cmI[i] = imp(np.transpose(cmD[i, :, :], (1, 0)))

    # print(f'Total number of evaluations: {numRun}')
    # return clI, cmI, clD, cmD

# def imp(X):
#     """Imposition of intervals."""
#     return np.array([np.max(X[0, :]), np.min(X[1, :])])

# # Placeholder for your xfoilrun function
# def simulate_xfoilrun(c):
#     """Simulates the xfoilrun function."""
#     # Replace this with your actual xfoilrun function or a suitable simulation
#     cl = c[0] + 2 * c[1] - 0.5 * c[2]  # Example calculation for cl
#     cm = c[1] * c[2] + c[3] / c[4]  # Example calculation for cm
#     return cl, cm



<a id="4.2.3.-Sampling-propagation"></a>

### 4.2.3. Sampling propagation

- Brute Monte Carlo
- Latin Hypercube
- Brute Monte Carlo + endspoints
- Latin Hypercuve + endpoints

In [ ]:
METHOD = "monte_carlo"
base_path = ""

a = epistemic_propagation(vars=['L', 'I', 'F', 'E'], 
          fun = cantilever_beam_deflection, 
          n = 1000, 
          method = METHOD, 
          save_raw_data = "no"
         )

In [ ]:
a.keys()

In [ ]:
a["un"]

<a id="4.2.4.-Optimisation-methors"></a>

### 4.2.4. Optimisation methods

- Local optimisation
- Genetic algorithm


In [ ]:
METHOD = "local_optimisation"

a = epistemic_propagation(vars=['L', 'I', 'F', 'E'], 
       fun = cantilever_beam_deflection, 
       x0 = None, 
       method = METHOD, 
       method_loc = 'Nelder-Mead')

In [ ]:
a.keys()

In [ ]:
a["un"]

In [ ]:
METHOD = "genetic_optimisation"

a = epistemic_propagation(vars=['L', 'I', 'F', 'E'], 
       fun = cantilever_beam_deflection, 
       method = METHOD)

In [ ]:
a.keys()

In [ ]:
a["un"]

### Naive interval arithmetics

In [ ]:
defl = cantilever_beam_deflection(L, I, F, E)

***
<a id="aleatory_uncertainty_propagation"></a>
# 5. Aleatory uncertainty

when inputs have various types of uncertainty.

In [ ]:
# y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) # Required only for stress estimation
L = UN(name='beam length', symbol='L', units='m', essence='distribution', distribution_parameters=["gaussian", [10.05, 0.033]])
I = UN(name='moment of inertia', symbol='I', units='m', essence='distribution', distribution_parameters=["gaussian", [0.000454, 4.5061e-5]])
F = UN(name='vertical force', symbol='F', units='kN', essence='distribution', distribution_parameters=["gaussian", [24, 8.67]])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='distribution', distribution_parameters=["gaussian", [210, 6.67]])

In [ ]:

METHOD = "monte_carlo"
base_path = ""

a = aleatory_propagation(vars=['L', 'I', 'F', 'E'], 
          fun = cantilever_beam_deflection, 
          n = 10, 
          method = METHOD, 
          save_raw_data = "no"
         )


***
<a id="mixed_uncertainty_propagation"></a>
# 6. Mixed types of uncertainty

when inputs have various types of uncertainty.

In [ ]:
#TODO could Scott elaborate? cartesian matrix distribution and interval or pbox - interval horizontal slices through the pbox define an interval, horizontal slicing as an approximation or rigolously each of the ci functions
#TODO ferson, hajagos rigorous
slicing approimately, slicling rigorously
a list of itnervals by horizontal slices and with these lists the cartesian product , interval from each input.
stack the intervals and reconstruct this is done. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def plotPbox(X, p=None, color='k'):
    """
    Plots a p-box (probability box) using matplotlib.

    Args:
        X (np.ndarray): A 2D NumPy array where each row represents an interval [xL, xR].
        p (np.ndarray, optional): A 1D NumPy array of probabilities corresponding to the intervals in X. 
                                   Defaults to None, which generates equally spaced probabilities.
        color (str, optional): The color of the plot. Defaults to 'k' (black).
    """
    xL = X[:, 0]
    xR = X[:, 1]

    if p is None:
        p = np.linspace(0, 1, X.shape[0] + 1)

    if p.min() > 0:
        p = np.concatenate(([0], p))
        xL = np.concatenate(([xL[0]], xL))
        xR = np.concatenate(([xR[0]], xR))

    if p.max() < 1:
        p = np.concatenate((p, [1]))
        xR = np.concatenate((xR, [xR[-1]]))
        xL = np.concatenate((xL, [xL[-1]]))

    plt.stairs([xL[0]] + xL.tolist(), p, color=color, linewidth=3)
    plt.stairs([xR[0]] + xR.tolist(), p, color=color, linewidth=3)
    plt.plot([xL[0], xR[0]], [0, 0], color=color, linewidth=3)
    plt.plot([xL[-1], xR[-1]], [1, 1], color=color, linewidth=3)
    plt.show()  # Add this to display the plot

In [ ]:
# example
from PyUncertainNumber.UP.mixed_uncertainty.second_order_propagation import second_order_propagation_method
def cantilever_beam_deflection(x):
        """Calculates deflection and stress for a cantilever beam.

        Args:
          x (np.array): Array of input parameters:
              x[0]: Length of the beam (m)
              x[1]: Second moment of area (mm^4)
              x[2]: Applied force (N)
              x[3]: Young's modulus (MPa)

      Returns:
          float: deflection (m)
                 Returns np.nan if calculation error occurs.
      """
        beam_length = x[0]
        I = x[1]
        F = x[2]
        E = x[3]
        try:  # try is used to account for cases where the input combinations leads to error in fun due to bugs
            deflection = F * beam_length**3 / (3 * E * 10**6 * I)  # deflection in m

        except:
            deflection = np.nan

        return deflection
    
# y = UncertainNumber(name='distance to neutral axis', symbol='y', units='m', essence='distribution', distribution_parameters=["gaussian", [0.15, 0.00333]])
# L = UncertainNumber(name='beam length', symbol='L', units='m', essence='distribution', distribution_parameters=["gaussian", [10.05, 0.033]])
# I = UncertainNumber(name='moment of inertia', symbol='I', units='m', essence='distribution', distribution_parameters=["gaussian", [0.000454, 4.5061e-5]])
# F = UncertainNumber(name='vertical force', symbol='F', units='kN', essence='distribution', distribution_parameters=["gaussian", [24, 8.67]])
# E = UncertainNumber(name='elastic modulus', symbol='E', units='GPa', essence='distribution', distribution_parameters=["gaussian", [210, 6.67]])
    
y = UN(name='beam width', symbol='y', units='m', essence='interval', bounds=[0.145, 0.155]) 
L = UN(name='beam length', symbol='L', units='m', essence='interval', bounds= [9.95, 10.05])
I = UN(name='moment of inertia', symbol='I', units='m', essence='interval', bounds= [0.0003861591, 0.0005213425])
F = UN(name='vertical force', symbol='F', units='kN', essence='interval', bounds= [11, 37])
E = UN(name='elastic modulus', symbol='E', units='GPa', essence='interval', bounds=[200, 220])

METHOD = "extremepoints"
base_path = "C:\\Users\\Ioanna\\OneDrive - The University of Liverpool\\DAWS2_code\\UP\\"
# (vars = None,
#         results:dict = None,
#         fun = None,
#         n: np.integer = None,
#         method = "monte_carlo", 
             
#         save_raw_data="no",
#         *,  # Keyword-only arguments start here
#         base_path=np.nan,
#         **kwargs,
a = second_order_propagation_method(vars= [ L, I, F, E], 
                            results = None,
                            fun= cantilever_beam_deflection, 
                            method= 'endpoints', 
                            #save_raw_data= "no",
                            save_raw_data= "yes",
                            base_path= base_path
                        )
    
    
a.print()
for i in range(len(a.un)):
    print(a.un[i])


    